# 1-Import library

In [1]:
import os
import datetime
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# 2- Import and wrangle data

In [2]:
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'

## Import Confirmed, Recovered and Deaths <br>

In [3]:
df_dict={}
for value_name in ['confirmed', 'deaths']:
    df = pd.read_csv(os.path.join(BASE_URL, 'time_series_covid19_{}_global.csv'.format(value_name)))

    df_value = df.drop(['Province/State', 'Lat', 'Long'], axis=1) \
                 .groupby(by='Country/Region') \
                 .agg('sum') \
                 .reset_index()

    date_list = df_value.columns.values.tolist()
    date_list.remove('Country/Region')
    df_value = pd.melt(df_value,
                       id_vars=['Country/Region'],
                       value_vars=date_list,
                       value_name=value_name,
                       var_name='date')
    df_value['date'] = pd.to_datetime(df_value['date'])

    df_dict[value_name] = df_value
    

## Merge and order data by country and date <br>

In [4]:
df = pd.merge(df_dict['confirmed'], df_dict['deaths'], on=['Country/Region', 'date'])
df.sort_values(by=['Country/Region', 'date'], inplace=True)

## Enrich with latitude and longitude of each country <br>

In [5]:
coordinate_dict = {'Australia': (-25, 133),
                   'Canada': (56, -106),
                   'China': (36, 104),
                   'Denmark': (55, 12),
                   'France': (46, 2),
                   'Netherlands': (52, 5),
                   'US': (37, -95),
                   'United Kingdom': (55, -3),
                  }

def coordinate_mod(df, coordinate_dict):
    if df['Country/Region'] in coordinate_dict:
        df['Lat'] = coordinate_dict[df['Country/Region']][0]
        df['Long'] = coordinate_dict[df['Country/Region']][1]
    return df

df_country = pd.read_csv(os.path.join(BASE_URL, 'time_series_covid19_confirmed_global.csv'))
df_country = df_country.groupby(by='Country/Region') \
                   .agg({'Lat':'first', 'Long': 'first', 'Province/State': 'first'}) \
                   .reset_index()

print(df_country.dropna()['Country/Region'])

df_country = df_country.apply(lambda x : coordinate_mod(x, coordinate_dict), axis=1) \
                       .drop(['Province/State'], axis=1)

df = pd.merge(df, df_country, on='Country/Region')

8           Australia
32             Canada
36              China
46            Denmark
61             France
120       Netherlands
175    United Kingdom
Name: Country/Region, dtype: object


## Enrich with continent for each country

In [6]:
continent = {'Afghanistan': 'Asia',
             'Albania': 'Europe',
             'Algeria': 'Africa',
             'Andorra': 'Europe',
             'Angola': 'Africa',
             'Antigua and Barbuda': 'North America',
             'Argentina': 'South America',
             'Armenia': 'Europe',
             'Australia': 'Oceania',
             'Austria': 'Europe',
             'Azerbaijan': 'Europe',
             'Bahamas': 'North America',
             'Bahrain': 'Asia',
             'Bangladesh': 'Asia',
             'Barbados': 'North America',
             'Belarus': 'Europe',
             'Belize': 'South America',
             'Belgium': 'Europe',
             'Benin': 'Africa',
             'Bhutan': 'Asia',
             'Bolivia': 'South America',
             'Bosnia and Herzegovina': 'Europe',
             'Botswana': 'Africa',
             'Brazil': 'South America',
             'Brunei': 'Asia',
             'Bulgaria': 'Europe',
             'Burkina Faso': 'Africa',
             'Burma': 'Asia',
             'Burundi': 'Africa',
             'Cabo Verde': 'Africa',
             'Cambodia': 'Asia',
             'Cameroon': 'Africa',
             'Canada': 'North America',
             'Central African Republic': 'Africa',
             'Chad': 'Africa',
             'Chile': 'South America',
             'China': 'Asia',
             'Colombia': 'South America',
             'Congo (Brazzaville)': 'Africa',
             'Congo (Kinshasa)': 'Africa',
             'Costa Rica': 'North America',
             "Cote d'Ivoire": 'Africa',
             'Croatia': 'Europe',
             'Cruise Ship': 'Asia',
             'Cuba': 'North America',
             'Cyprus': 'Europe',
             'Czechia': 'Europe',
             'Denmark': 'Europe',
             'Diamond Princess': 'Ship',
             'Djibouti': 'Africa',
             'Dominica': 'North America',
             'Dominican Republic': 'North America',
             'Ecuador': 'South America',
             'Egypt': 'Africa',
             'El Salvador': 'South America',
             'Equatorial Guinea': 'Africa',
             'Eritrea': 'Africa',
             'Estonia': 'Europe',
             'Eswatini': 'Africa',
             'Ethiopia': 'Europe',
             'Fiji': 'Oceania',
             'Finland': 'Europe',
             'France': 'Europe',
             'Gabon': 'Africa',
             'Gambia': 'Africa',
             'Georgia': 'Europe',
             'Germany': 'Europe',
             'Ghana': 'Africa',
             'Greece': 'Europe',
             'Greenland': 'North America',
             'Grenada': 'North America',
             'Guatemala': 'North America',
             'Guinea': 'Africa',
             'Guinea-Bissau': 'Africa',
             'Guyana': 'South America',
             'Haiti': 'North America',
             'Holy See': 'Europe',
             'Honduras': 'South America',
             'Hungary': 'Europe',
             'Iceland': 'Europe',
             'India': 'Asia',
             'Indonesia': 'Asia',
             'Iran': 'Asia',
             'Iraq': 'Asia',
             'Ireland': 'Europe',
             'Israel': 'Asia',
             'Italy': 'Europe',
             'Jamaica': 'North America',
             'Japan': 'Asia',
             'Jordan': 'Asia',
             'Kazakhstan': 'Asia',
             'Kenya': 'Africa',
             'Korea, South': 'Asia',
             'Kosovo': 'Europe',
             'Kuwait': 'Asia',
             'Kyrgyzstan': 'Asia',
             'Laos': 'Asia',
             'Latvia': 'Europe',
             'Lebanon': 'Asia',
             'Liberia': 'Africa',
             'Libya': 'Africa',
             'Liechtenstein': 'Europe',
             'Lithuania': 'Europe',
             'Luxembourg': 'Europe',
             'MS Zaandam': 'Ship', 
             'Madagascar': 'Africa',
             'Malawi': 'Africa',
             'Malaysia': 'Asia',
             'Mali': 'Africa',
             'Maldives': 'Asia',
             'Malta': 'Europe',
             'Martinique': 'North America',
             'Mauritania': 'Africa',
             'Mauritius': 'Africa',
             'Mexico': 'North America',
             'Moldova': 'Europe',
             'Monaco': 'Europe',
             'Mongolia': 'Asia',
             'Montenegro': 'Europe',
             'Morocco': 'Africa',
             'Mozambique': 'Africa',
             'Namibia': 'Africa',
             'Nepal': 'Asia',
             'Netherlands': 'Europe',
             'New Zealand': 'Oceania',
             'Nicaragua': 'South America',
             'Niger': 'Africa',
             'Nigeria': 'Africa',
             'North Macedonia': 'Europe',
             'Norway': 'Europe',
             'Oman': 'Africa',
             'Pakistan': 'Asia',
             'Panama': 'North America',
             'Papua New Guinea': 'Asia',
             'Paraguay': 'South America',
             'Peru': 'South America',
             'Philippines': 'Asia',
             'Poland': 'Europe',
             'Portugal': 'Europe',
             'Qatar': 'Asia',
             'Romania': 'Europe',
             'Russia': 'Europe',
             'Rwanda': 'Africa',
             'Saint Kitts and Nevis': 'North America',
             'Saint Lucia': 'North America',
             'Saint Vincent and the Grenadines': 'North America',
             'San Marino': 'Europe',
             'Sao Tome and Principe': 'Africa',
             'Saudi Arabia': 'Asia',
             'Senegal': 'Africa',
             'Serbia': 'Europe',
             'Seychelles': 'Africa',
             'Sierra Leone': 'Africa',
             'Singapore': 'Asia',
             'Slovakia': 'Europe',
             'Slovenia': 'Europe',
             'Somalia': 'Africa',
             'South Africa': 'Africa',
             'South Sudan': 'Africa',
             'Spain': 'Europe',
             'Sri Lanka': 'Africa',
             'Sudan': 'Africa',
             'Suriname': 'South America',
             'Sweden': 'Europe',
             'Switzerland': 'Europe',
             'Syria': 'Africa',
             'Taiwan*': 'Asia',
             'Tanzania': 'Africa',
             'Thailand': 'Asia',
             'The Bahamas': 'North America',
             'Timor-Leste': 'Asia',
             'Togo': 'Africa',
             'Trinidad and Tobago': 'North America',
             'Tunisia': 'Africa',
             'Turkey': 'Europe',
             'US': 'North America',
             'Uganda': 'Africa',
             'Ukraine': 'Europe',
             'United Arab Emirates': 'Asia',
             'United Kingdom': 'Europe',
             'Uruguay': 'South America',
             'Uzbekistan': 'Asia',
             'Venezuela': 'South America',
             'Vietnam': 'Asia',
             'West Bank and Gaza':'Asia', 
             'Western Sahara': 'Africa',
             'Zambia': 'Africa',
             'Zimbabwe': 'Africa'
              }

df['continent'] = df['Country/Region'].map(continent)

In [7]:
df[df['continent'].isnull()]['Country/Region'].unique()

array([], dtype=object)

## Smooth, derive and enrich with mortality

In [8]:
def smooth_data(df, param):
    try:
        df[param] = savgol_filter(df[param] , 7, 3)
    except:
        pass
    return df

In [9]:
def derive_data(df, param):
    try:
        df[param + '_derive'] = savgol_filter(df[param] , 7, 3, deriv=1)
    except:
        df[param + '_derive'] = df[param] - df[param].shift(1)
    return df

In [10]:
def confirmed_computed(df):
    df['confirmed_computed'] = df['deaths'].shift(-6, fill_value=0) / 0.01
    df['confirmed_ratio'] = 10.
    for i in df.index:
        if (df.loc[i, 'confirmed_computed']==0) & (i>df.index.min()):
            df.loc[i, 'confirmed_ratio'] = df.loc[i-1, 'confirmed_computed'] / df.loc[i-1, 'confirmed']
            df.loc[i, 'confirmed_computed'] = df.loc[i-1, 'confirmed_computed'] + df.loc[i, 'confirmed_derive'] * df.loc[i,'confirmed_ratio']
    return df

In [11]:
for param in ['confirmed', 'deaths']:
    df = df.groupby('Country/Region').apply(lambda x: smooth_data(x, param))
    df = df.groupby('Country/Region').apply(lambda x: derive_data(x, param))

In [12]:
df = df.groupby('Country/Region').apply(lambda x: confirmed_computed(x))
df = df.groupby('Country/Region').apply(lambda x: smooth_data(x, 'confirmed_computed'))
df = df.groupby('Country/Region').apply(lambda x: derive_data(x, 'confirmed_computed'))

/Users/thomas/anaconda3/envs/corona_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [13]:
df['mortality'] = df['deaths'] / df['confirmed'] * 100

## Enrich with patient zero and confinement date for each country

In [14]:
patient_zero = {'China': pd.to_datetime("2020-01-06"),
               'Italy': pd.to_datetime("2020-02-11"),
               'Iran': pd.to_datetime("2020-02-12"),
               'Korea, South': pd.to_datetime("2020-02-09"),
               'Spain': pd.to_datetime("2020-02-18"),
               'France': pd.to_datetime("2020-02-22"),
               'Germany': pd.to_datetime("2020-03-01"),
               'US': pd.to_datetime("2020-02-24"),
               'United Kingdom': pd.to_datetime("2020-02-26"),
               'Netherlands': pd.to_datetime("2020-02-29"),
               'Belgium': pd.to_datetime("2020-03-02"),
               'Switzerland': pd.to_datetime("2020-03-01"),
               'Brazil': pd.to_datetime("2020-03-06"),
               'Turkey': pd.to_datetime("2020-03-06"),
               'Sweden': pd.to_datetime("2020-03-06"),
               'Indonesia': pd.to_datetime("2020-03-06"),
               'Portugal': pd.to_datetime("2020-03-07"),
               'Philippines': pd.to_datetime("2020-03-06"),
               'Austria': pd.to_datetime("2020-03-08"),
               'Denmark': pd.to_datetime("2020-03-08"),
              }

confinement = {'China': pd.to_datetime("2020-01-23"),
               'Italy': pd.to_datetime("2020-03-08"),
               'Spain': pd.to_datetime("2020-03-14"),
               'France': pd.to_datetime("2020-03-16"),
               'Germany': pd.to_datetime("2020-03-22"),
               'US': pd.to_datetime("2020-03-20"),
               'United Kingdom': pd.to_datetime("2020-03-23"),
               'Netherlands': pd.to_datetime("2020-03-23"),
               'Belgium': pd.to_datetime("2020-03-18"),
               'Switzerland': pd.to_datetime("2020-03-16"),
               'Turkey': pd.to_datetime("2020-03-27"),
               'Indonesia': pd.to_datetime("2020-03-29"),
               'Portugal': pd.to_datetime("2020-03-20"),
               'Philippines': pd.to_datetime("2020-03-16"),
               'Austria': pd.to_datetime("2020-03-16"),
               'Denmark': pd.to_datetime("2020-03-11"),
              }

df['patient_zero'] = df['Country/Region'].map(patient_zero)
df['confinement'] = df['Country/Region'].map(confinement)
df['contagion_days'] = (df['date'] - df['patient_zero']) / pd.to_timedelta(1, unit='D')
df['contagion_days_confinement'] = (df['confinement'] - df['patient_zero']) / pd.to_timedelta(1, unit='D')

## Display some data

In [15]:
df.head()

,Country/Region,date,confirmed,deaths,Lat,Long,continent,confirmed_derive,deaths_derive,confirmed_computed,confirmed_ratio,confirmed_computed_derive,mortality,patient_zero,confinement,contagion_days,contagion_days_confinement
0,Afghanistan,2020-01-22,0.0,0.0,33.0,65.0,Asia,0.0,0.0,0.0,10.0,NaN,NaN,NaT,NaT,NaN,NaN
1,Afghanistan,2020-01-23,0.0,0.0,33.0,65.0,Asia,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
2,Afghanistan,2020-01-24,0.0,0.0,33.0,65.0,Asia,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
3,Afghanistan,2020-01-25,0.0,0.0,33.0,65.0,Asia,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
4,Afghanistan,2020-01-26,0.0,0.0,33.0,65.0,Asia,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


### List of countries

In [16]:
df['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guinea', 'Guine

### List of countries with max deaths

In [17]:
df.groupby(by='Country/Region') \
  .agg({'deaths':'max'}) \
  .reset_index() \
  .sort_values(by='deaths', ascending=False) \
  .reset_index() \
  .head(10)

,index,Country/Region,deaths
0,84,Italy,17123.142857
1,156,Spain,14046.357143
2,171,US,12657.023810
3,61,France,10289.119048
4,175,United Kingdom,6126.142857
5,80,Iran,3870.523810
6,36,China,3335.119048
7,120,Netherlands,2095.238095
8,16,Belgium,2024.571429
9,65,Germany,2020.357143


# 3-Data analysis

### Analysis by continent and top 10 countries in each continent

In [18]:
def plot_contagion_by_continent(plot_type, analysis, param):
    if analysis == 'Continent':
        group = 'continent'
        mask = pd.Series(True, index=np.arange(df.shape[0]))
    if analysis == 'World':
        group = 'Country/Region'
        country_list = df.groupby(by='Country/Region') \
                         .agg({'deaths':'max'}) \
                         .sort_values(by='deaths', ascending=False) \
                         .head(10) \
                         .index
        mask = df['Country/Region'].isin(country_list)
    if analysis in ['Asia', 'Europe', 'North America', 'South America', 'Africa', 'Oceania']:
        group = 'Country/Region'
        country_list = df[df['continent']==analysis].groupby(by='Country/Region') \
                         .agg({'deaths':'max'}) \
                         .sort_values(by='deaths', ascending=False) \
                         .head(10) \
                         .index
        mask = df['Country/Region'].isin(country_list)       

    df_tmp = df[mask].groupby(by=['date', group]) \
               .agg({param: 'sum'}).reset_index() \
               .pivot(index='date', columns=group, values= param)
    for var in df_tmp.columns:
        df_tmp[var] = df_tmp[var].apply(lambda x: x if x>0 else 0)
    
    if plot_type == 'Area':
        ax = df_tmp.plot(kind='area', grid=True)
    if plot_type == 'Line':
        ax = df_tmp.plot(kind='line', grid=True)

    
selection_plot = widgets.Dropdown(
    options=['Area', 'Line'],
    value='Area',
    description='Plot type',
    disabled=False,
)

selection_group = widgets.Dropdown(
    options=['Continent', 'World', 'Asia', 'Europe', 'North America', 'South America', 'Africa', 'Oceania'],
    value='Continent',
    description='Analysis',
    disabled=False,
)

options=['confirmed', 'confirmed_computed', 'deaths', 'confirmed_derive', 'confirmed_computed_derive', 'deaths_derive']

selection_param = widgets.Dropdown(
    options=options,
    value=options[0],
    description='Param',
    disabled=False,
)

interact(plot_contagion_by_continent,
         plot_type = selection_plot,
         analysis = selection_group,
         param=selection_param)

interactive(children=(Dropdown(description='Plot type', options=('Area', 'Line'), value='Area'), Dropdown(desc…

<function __main__.plot_contagion_by_continent(plot_type, analysis, param)>

### Confirmed, Deaths, Recovered and mortality by country

In [19]:
def plot_contagion_by_country(country, param_list):
    mask = df['Country/Region'] == country
    if 'mortality' in param_list:
        param_list = list(param_list)
        param_list.remove('mortality')
        if param_list != []:
            ax1 = df[mask].plot(x='date', y=list(param_list), grid=True)
            ax1.legend(loc='upper left')
            ax2 = ax1.twinx()
            df[mask].plot(x='date', y=['mortality'], c='k', linestyle='--', ax=ax2)
            ax2.legend(loc='upper right')
        else:
            ax1 = df[mask].plot(x='date', y='deaths', linewidth=0, grid=True)
            ax1.legend().remove()
            ax2 = ax1.twinx()
            df[mask].plot(x='date', y=['mortality'], c='k', linestyle='--', ax=ax2)
            ax2.legend(loc='upper right')
    else:
        param_list = list(param_list)
        ax1 = df[mask].plot(x='date', y=list(param_list), grid=True)
        ax1.legend(loc='upper left')


selection_slider = widgets.Dropdown(
    options=df['Country/Region'].unique(),
    value='France',
    description='Country',
    disabled=False,
)

options=['confirmed', 'confirmed_computed', 'deaths', 'confirmed_derive', 'confirmed_computed_derive', 'deaths_derive', 'mortality']

selection_list = widgets.SelectMultiple(
    options=options,
    value=['confirmed', 'deaths', 'mortality'],
    description='Param',
    rows=len(options),
    disabled=False
)

interact(plot_contagion_by_country, country = selection_slider, param_list=selection_list)

interactive(children=(Dropdown(description='Country', index=61, options=('Afghanistan', 'Albania', 'Algeria', …

<function __main__.plot_contagion_by_country(country, param_list)>

### Comparison of Confirmed, Deaths, Recovered and mortality between countries

In [20]:
def plot_compare_contagion(confinement_date, paramx, param1, param2, countries):
    if paramx == 'date':
        start_date = datetime.date.today() + datetime.timedelta(-60)
        end_date = datetime.date.today()
        xlim=(start_date, end_date)
    if paramx == 'contagion_days':
        xlim=(0, 60)
    fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8, 6))
    for country in countries:
        mask = df['Country/Region'] == country
        df[mask].plot(x=paramx, y=param1, label=country, xlim=xlim, grid=True, ax=ax[0])
        df[mask].plot(x=paramx, y=param2, label=country, grid=True, ax=ax[1])
        
        if confinement_date:
            
            if paramx == 'date':
                confinement_date = df[mask]['confinement'].min()
                if type(confinement_date) is pd.Timestamp:
                    ax[0].axvline(x=confinement_date, color=plt.gca().lines[-1].get_color(), linestyle='--')
                    ax[1].axvline(x=confinement_date, color=plt.gca().lines[-1].get_color(), linestyle='--')

            if paramx == 'contagion_days':
                contagion_days_confinement = df[mask]['contagion_days_confinement'].min()
                if not math.isnan(contagion_days_confinement):
                    ax[0].axvline(x=contagion_days_confinement, color=plt.gca().lines[-1].get_color(), linestyle='--')
                    ax[1].axvline(x=contagion_days_confinement, color=plt.gca().lines[-1].get_color(), linestyle='--')

    handles, labels = ax[0].get_legend_handles_labels()
    ax[0].legend().remove()
    ax[1].legend().remove()
    ax[0].set_ylabel(param1)
    ax[1].set_ylabel(param2)
    fig.legend(handles, labels, bbox_to_anchor=(1.3, 0.9))
    plt.tight_layout()
    
selection_confinement = widgets.Checkbox(
    value=False,
    description='Confinement date',
    indent=True
)

selection_param_x = widgets.Dropdown(
    options=['date', 'contagion_days'],
    value='date',
    description='Paramx',
    disabled=False,
)

options=['confirmed', 'confirmed_computed', 'deaths', 'confirmed_derive', 'confirmed_computed_derive', 'deaths_derive']

selection_param_1 = widgets.Dropdown(
    options=options,
    value=options[0],
    description='Param1',
    disabled=False,
)

selection_param_2 = widgets.Dropdown(
    options=options,
    value=options[2],
    description='Param2',
    disabled=False,
)

selection_list = widgets.SelectMultiple(
    options=df['Country/Region'].unique(),
    value=list(patient_zero),
    description='Countries',
    rows=10,
    disabled=False
)

interact(plot_compare_contagion,
         confinement_date=selection_confinement,
         paramx=selection_param_x,
         param1=selection_param_1,
         param2=selection_param_2,
         countries = selection_list)


interactive(children=(Checkbox(value=False, description='Confinement date'), Dropdown(description='Paramx', op…

<function __main__.plot_compare_contagion(confinement_date, paramx, param1, param2, countries)>

### Model

In [21]:
def plot_with_model(country, m1, m2, delta_days):
    n=3
    patient_zero_date=patient_zero[country]
    if country in confinement:
        confinement_date=confinement[country]
    periods=60/n

    df_tmp=pd.DataFrame()
    df_tmp['date'] = pd.date_range(patient_zero_date, periods=periods, freq='{}d'.format(n))

    df_tmp.loc[0, 'confirmed_model'] = 300
    df_tmp.loc[0, 'confirmed_model_with_confinement'] = 300
    df_tmp.loc[1, 'confirmed_model'] = 600
    df_tmp.loc[1, 'confirmed_model_with_confinement'] = 600

    for i in range(2, len(df_tmp)):
        df_tmp.loc[i, 'confirmed_model'] = (df_tmp.loc[i-1, 'confirmed_model'] - df_tmp.loc[i-2, 'confirmed_model'])* m1 + df_tmp.loc[i-1, 'confirmed_model']
        if country in confinement: 
            if df_tmp.loc[i, 'date']<confinement_date + datetime.timedelta(delta_days):
                df_tmp.loc[i, 'confirmed_model_with_confinement'] = (df_tmp.loc[i-1, 'confirmed_model_with_confinement'] - df_tmp.loc[i-2, 'confirmed_model_with_confinement']) * m1 + df_tmp.loc[i-1, 'confirmed_model_with_confinement']
            else:
                df_tmp.loc[i, 'confirmed_model_with_confinement'] = (df_tmp.loc[i-1, 'confirmed_model_with_confinement'] - df_tmp.loc[i-2, 'confirmed_model_with_confinement']) * m2 + df_tmp.loc[i-1, 'confirmed_model_with_confinement']

    df_tmp['deaths_model'] = df_tmp['confirmed_model'].shift(2)*0.01
    df_tmp['deaths_model_with_confinement'] = df_tmp['confirmed_model_with_confinement'].shift(2)*0.01

    mask = df['Country/Region'] == country
    fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8, 6))
    df[mask].plot(x='date', y=['confirmed', 'confirmed_computed'], grid=True, ax=ax[0])
    df_tmp.plot(kind='line',
                 x='date',
                 y=['confirmed_model', 'confirmed_model_with_confinement'],
                 grid=True, ax=ax[0])
    ax[0].set_ylim(0, 1.5*df[mask]['confirmed_computed'].max())
    
    df[mask].plot(x='date', y='deaths', grid=True, ax=ax[1])
    df_tmp.plot(kind='line',
                 x='date',
                 y=['deaths_model', 'deaths_model_with_confinement'],
                 grid=True, ax=ax[1])
    ax[1].set_ylim(0, 1.5*df[mask]['deaths'].max())

options = list(patient_zero)
options.sort()
    
selection_slider = widgets.Dropdown(
    options=options,
    value='France',
    description='Country',
    disabled=False,
)

selection_slider_m1 = widgets.FloatSlider(
    value=2.0,
    min=1.0,
    max=3.0,
    step=0.1,
    description='R0 before confinement:',
    layout={'width': '400px'},
    style = {'description_width': 'initial'}
)

selection_slider_m2 = widgets.FloatSlider(
    value=1.3,
    min=1.0,
    max=3.0,
    step=0.1,
    description='R0 after confinement:',
    layout={'width': '400px'},
    style = {'description_width': 'initial'}, 
)

selection_slider_delta_days = widgets.IntSlider(
    value=6,
    min=0,
    max=15,
    step=3,
    description='confinement delay:',
    layout={'width': '400px'},
    style = {'description_width': 'initial'},
)

interact(plot_with_model,
         country=selection_slider,
         m1=selection_slider_m1,
         m2=selection_slider_m2,
         delta_days=selection_slider_delta_days,
        )


interactive(children=(Dropdown(description='Country', index=5, options=('Austria', 'Belgium', 'Brazil', 'China…

<function __main__.plot_with_model(country, m1, m2, delta_days)>